<a href="https://colab.research.google.com/github/harry-kav/BirdCount-CV/blob/main/Copy_of_birds_kerascv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

def fixBadZipfile(zipFile):
  f = open(zipFile, 'r+b')
  data = f.read()
  pos = data.find('\x50\x4b\x05\x06') # End of central directory signature
  if (pos > 0):
    self._log("Truncating file at location " + str(pos + 22)+ ".")
    f.seek(pos + 22)   # size of 'ZIP end of central directory record'
    f.truncate()
    f.close()
  else:
     raise ValueError("couldn't fix error")

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'au-bird-scene-photo-collection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F679337%2F1193435%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240522%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240522T113305Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8ac2bd81540499fc1210526886268c4b8c0e85c5b58881b10d3e706a82feb952e8514d0facd722803ccb8eef6e4c0323e51691c195f53ff7a5eabffbf2986de0e2a4adb32ada61e924148d5d3c49df2bebe7b5de3d981444cd152c62bb62a1fcc2c6c95fd38fad202ed5256d20cba277ca2f4ae3ce10e08ebe1a71804f269ac09371cc204fb85e033c2ba60b52c5154553edccc05af58cbf4240c327bee0dffbbf741c05f2c1837a85e0b9eebb1dbe2ea9a5d17c68cec48b139b21db92e68debc7cb161b072ffd87d4fad4607aaa9bcbe0dde551c7e9487ec087b119abc63c63962c778161538f77adf09abd61408472dcc488a7cab68d12a2c52ad45114dab0,yolov8/keras/yolo_v8_l_backbone_coco/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4651%2F6108%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240522%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240522T113305Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D008707cbe70119d16db4aa2d9b9583da88302e3b4106fa6e7e2c029a18da10d6b1ae80b592d6ba5b5cf73ce6b4218f75716c74b7d1cc5194d5176ddf4d9f758f6603afcc58bfc7fe3776e85b7935449f396b29e0ee6949a24166efecdc3d8e004a9f645f08cf30cf1e99dea27f0d701afad195444223c7d3b70da61a6dbb16349462bf530c37ca9bc6d6bfba435caaecb5d375797768876802aef5155ca946a6b8eecf6826ea4de8abf38193d5301189ece047e9f9233bb549d2d81a8b470227eb057103fa5d931930d1656f23a9bd8b37352cd8e4e5330ae50470e55312103f04984bb3cacc04303afdcbc1c443f9eaa74a435765af80084a878b11f6ffb1e6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              tries = 3
              i = 0
              while i < tries:
                try:
                  with ZipFile(tfile) as zfile:
                    zfile.extractall(destination_path)
                    break
                except:
                  fixBadZipfile(tfile)
                  i = i + 1
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[====                                              ] 1811169280 bytes downloaded

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow

In [ ]:
#!pip install --upgrade keras-cv tensorflow
#!pip install --upgrade keras
#!pip install pycocotools
!pip install keras-cv==0.5.1
!pip install keras-core

#!pip install -q --upgrade keras-cv tensorflow
#!pip install -q --upgrade keras  # Upgrade to Keras 3.

#!pip install git+https://github.com/keras-team/keras-cv.git tensorflow --upgrade --quiet
#!pip install pycocotools --quiet

In [ ]:
#!pip install --upgrade git+https://github.com/keras-team/keras-cv -q

In [ ]:
import os
import xml.etree.ElementTree as ET
import tensorflow as tf

In [ ]:
import os
import xml.etree.ElementTree as ET
import tensorflow as tf
import keras_cv
import requests
import zipfile

from tqdm.auto import tqdm
from tensorflow import keras
from keras_cv import bounding_box
from keras_cv import visualization

In [ ]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 8
LEARNING_RATE = 0.001
EPOCH = 50
GLOBAL_CLIPNORM = 10.0 # 1, 3, 5, 8, 10

In [ ]:
class_ids = [
    "bird"
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))


train_folder = '/kaggle/input/au-bird-scene-photo-collection/AUbirds/train'
test_folder = '/kaggle/input/au-bird-scene-photo-collection/AUbirds/train'

def get_files(path, extension=".xml"):

    # Get all XML file paths in path_annot and sort them
    files = sorted(
        [
            os.path.join(path, file_name)
            for file_name in os.listdir(path)
            if file_name.endswith(extension)
        ]
    )
    return files

train_xml_files = get_files(train_folder, ".xml")
test_xml_files = get_files(test_folder, ".xml")
train_jpg_files = get_files(train_folder, ".jpg")
test_jpg_files = get_files(test_folder, ".jpg")



In [ ]:
def parse_annotation(xml_file,folder):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(folder, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids

#train
train_image_paths = []
train_bbox = []
train_classes = []
for xml_file in tqdm(train_xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file,train_folder)
    train_image_paths.append(image_path)
    train_bbox.append(boxes)
    train_classes.append(class_ids)


#test
test_image_paths = []
test_bbox = []
test_classes = []
for xml_file in tqdm(test_xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file,test_folder)
    test_image_paths.append(image_path)
    test_bbox.append(boxes)
    test_classes.append(class_ids)

In [ ]:
train_val_bbox = tf.ragged.constant(train_bbox)
train_val_classes = tf.ragged.constant(train_classes)
train_val_image_paths = tf.ragged.constant(train_image_paths)

train_val_data = tf.data.Dataset.from_tensor_slices((train_val_image_paths, train_val_classes, train_val_bbox))

test_bbox = tf.ragged.constant(test_bbox)
test_classes = tf.ragged.constant(test_classes)
test_image_paths = tf.ragged.constant(test_image_paths)

test_data = tf.data.Dataset.from_tensor_slices((test_image_paths, test_classes, test_bbox))

In [ ]:
# Determine the number of validation samples
num_val = int(len(train_xml_files) * SPLIT_RATIO)

# Split the dataset into train and validation sets
val_data = train_val_data.take(num_val)
train_data = train_val_data.skip(num_val)

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}

augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xyxy"),
        keras_cv.layers.JitteredResize(
            target_size=(640,640),
            scale_factor=(1.0, 1.0),
            bounding_box_format="xyxy",
        ),
    ]
)

train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)

resizing = keras_cv.layers.JitteredResize(
    target_size=(640,640),
    scale_factor=(1.0, 1.0),
    bounding_box_format="xyxy",
)

inference_resizing = keras_cv.layers.Resizing(
    640, 640, bounding_box_format="xyxy", pad_to_aspect_ratio=True
)

val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)

test_ds = test_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(BATCH_SIZE * 4)
test_ds = test_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
test_ds = test_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )


visualize_dataset(
    train_ds, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2
)

visualize_dataset(
    val_ds, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2
)

In [ ]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]


train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_l_backbone_coco",
    load_weights=True
)

model = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=3,
)

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

model.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)

In [ ]:
class EvaluateCOCOMetricsCallback(keras.callbacks.Callback):
    def __init__(self, data, save_path):
        super().__init__()
        self.data = data
        self.metrics = keras_cv.metrics.BoxCOCOMetrics(
            bounding_box_format="xyxy",
            evaluate_freq=1e9,
        )

        self.save_path = save_path
        self.best_map = -1.0

    def on_epoch_end(self, epoch, logs):
        self.metrics.reset_state()
        for batch in self.data:
            images, y_true = batch[0], batch[1]
            y_pred = self.model.predict(images, verbose=0)
            self.metrics.update_state(y_true, y_pred)

        metrics = self.metrics.result(force=True)
        logs.update(metrics)

        current_map = metrics["MaP"]
        if current_map > self.best_map:
            self.best_map = current_map
            self.model.save(self.save_path)  # Save the model when mAP improves

        return logs

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="kaggle/working")

coco_metrics_callback = keras_cv.callbacks.PyCOCOCallback(
    eval_ds, bounding_box_format="xywh"
)


early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   # Monitor the validation loss
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,            # To display messages when the callback takes an action
    mode='min',           # Stops training when the quantity monitored stops decreasing
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity
)

In [ ]:
#!pip install Cython
#!pip install pycocotools
#!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
#!pip install /kaggle/input/pycocotools-whl/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl


In [ ]:
#!pip list

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCH,
    callbacks=[
        EvaluateCOCOMetricsCallback(val_ds, "model_yolov8large.h5"),
        tensorboard_callback,
    ],
)